# Performing Inference

In [ ]:
import jax
import jax.numpy as jnp
from jax.sharding import Mesh
from jax.sharding import NamedSharding

import flax

from rich import print
import numpy as np
import os

from torch_to_flax import torch_to_flax
from model_flax import get_partition_rules, Qwen2Config, Qwen2ForCausalLM

# Set this environment variable before importing JAX.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9375"


SHARD_MODEL = True


def create_sharding_specs(params):

    # Get available JAX devices and create mesh
    devices = jax.devices()
    device_mesh = np.array(devices).reshape(-1)  # 1D mesh
    mesh = Mesh(device_mesh, ("mp",))
    print(mesh)

    partition_rules = get_partition_rules()

    def assign_spec(path, _):
        # Create a slash-separated string from the tuple path
        path_str = "/".join(map(str, path))
        # Look for a matching partition rule
        for rule_path, spec in partition_rules:
            if rule_path in path_str:
                return NamedSharding(mesh, spec)
        # If no rule matches, return a default sharding spec
        return NamedSharding(mesh, None)

    
    return jax.tree_util.tree_map_with_path(assign_spec, params)


def init_model(SHARD_MODEL:bool):
    """Initialize a model with parameters on CPU then shard them to GPU after checkpoint loading."""
    # Create config and model
    config = Qwen2Config()
    model = Qwen2ForCausalLM(config=config)

    rng = jax.random.PRNGKey(0)
    input_shape = (1, 32)

    # Force initialization on CPU to avoid duplicate GPU allocations
    with jax.default_device(jax.devices("cpu")[0]):
        try:
            params = model.init(rng, jnp.ones(input_shape, dtype=jnp.int4))
        except Exception as e:
            params = model.init(rng, jnp.ones(input_shape, dtype=jnp.int32))


    # Load the parameters from the file
    try:
        with open("flax_params.msgpack", "rb") as f:
            params = {
                "params": flax.serialization.from_bytes(params["params"], f.read())
            }
    except FileNotFoundError:
        print("File not found. Running conversion...")
        torch_to_flax()
        with open("flax_params.msgpack", "rb") as f:
            params = {
                "params": flax.serialization.from_bytes(params["params"], f.read())
            }

    # Shard the parameters
    if SHARD_MODEL:
        sharding_specs = create_sharding_specs(params)

        print(sharding_specs)

        params = jax.tree_util.tree_map(
            lambda x, spec: jax.device_put(x, spec), params, sharding_specs
        )
    else:
        params = jax.device_put(params)


    return model, params

model, params = init_model(SHARD_MODEL)
print("Sharded model initialized.")


: 

In [2]:
from transformers import AutoTokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)


prompt = "What is 3 + 4? <think>\n"
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = jnp.array(inputs["input_ids"].numpy())
# Generate
print("Generating tokens...")
output = model.generate(
    params,
    input_ids,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True,
    prng_key=jax.random.PRNGKey(0),
)

# Decode using your tokenizer
decoded = tokenizer.decode(np.array(output[0]))
print("Decoded text:", decoded)

Generating tokens...

100%|██████████| 100/100 [00:14<00:00,  6.90it/s]


Decoded text: <｜begin▁of▁sentence｜>What is 3 + 4? <think>
To solve 3 plus 4, I start by identifying the numbers involved, which are 3 and 4.

Next, I add these two numbers together by combining them: 3 and 4 make 7.

Therefore, the final answer is 7.
</think>

To solve the addition \(3 + 4\), follow these easy steps:

1. **Identify the numbers to add:**
   [
   3 \quad \text{and} \quad 4